In [26]:
import sympy as sp
import random

import symqups
from symqups import s as CahillGlauberS
from symqups import *
from symqups._internal.multiprocessing import *
from symqups.objects.scalars import *
from symqups.objects.operators import *
from symqups.manipulations import *
from symqups.ordering import *
from symqups.quantization import *
from symqups.cg import *
from symqups.objects import *
from symqups._internal.cache import *
from symqups.star_product import *
from symqups._internal.basic_routines import operation_routine
from symqups._internal.operator_handling import (
    separate_operator,
    separate_term_by_polynomiality,
    separate_term_oper_by_sub,
    collect_alpha_type_oper_from_monomial_by_sub,
    get_oper_sub
)
from symqups._internal.warnings import suppress_warning_repeats
from symqups._internal.grouping import UnBoppable, NotAnOperator, PhaseSpaceObject
from symqups.utils import get_random_poly
# from symqups.eom import LindbladMasterEquation
from typing import Sequence

# a = [alpha(i) for i in range(10)]
# aop = [annihilateOp(i) for i in range(10)]
# ad = [alphaD(i) for i in range(10)]
# adop = [createOp(i) for i in range(10)]
# x = sp.Symbol("x")
# qq = [q(i) for i in range(10)]
# pp = [p(i) for i in range(10)]
CahillGlauberS.val = random.random()
# # symqups.hbar.val = sp.Number(1)
# CahillGlauberS
a = alpha()
aop = annihilateOp()
ad = alphaD()
adop = createOp()
pi = pi.val

In [ ]:
CGTransform(aop*rho/pi + )

\alpha_{}*StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}) + 0.581784751941383*Derivative(StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}), \overline{\alpha}_{})

In [16]:
CGTransform(rho*ad)

pi*\overline{\alpha}_{}*StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}) - pi*Derivative(StateFunction(t_{}, \alpha_{}, \overline{\alpha}_{}), \alpha_{})/2